In [2]:
# import necessary packages
import sys
import requests
import re
import json
from requests.exceptions import RequestException
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import pyecharts

In [2]:
# head information to pretend like a broswer
# User-Agent can be acquired by opening a normal webpage: F12--Network(F5)--Request Header(user-agent)
headers = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding':'gzip, deflate, sdch',
    'Accept-Language':'zh-CN,zh;q=0.8',
    'Cache-Control':'max-age=0',
    'Connection':'keep-alive',
    'Cookie':'gameType=2; wanplus_token=99741ab201410ab6ecdb281a7b5dd663; wanplus_storage=lf4m67eka3o; wanplus_sid=582f6a3bb9f54d26823669a8ea21764e; wanplus_csrf=_csrf_tk_812504310; Hm_lvt_f69cb5ec253c6012b2aa449fb925c1c2=1494295646; Hm_lpvt_f69cb5ec253c6012b2aa449fb925c1c2=1494296667',
    'Host':'www.wanplus.com',
    'Upgrade-Insecure-Requests':'1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
}
#url_team = 'https://www.wanplus.com/lol/event/758/team' # spring LPL
#url_player = 'https://www.wanplus.com/lol/event/758/player'# spring LPL

url_team = 'https://www.wanplus.com/lol/event/817/team' # summer LPL
url_player = 'https://www.wanplus.com/lol/event/817/player'# summer LPL

teamdata_head = ['名次','战队','KDA','场均击杀','场均死亡','每分钟伤害','一血率','场均时长','场均经济','每分钟经济','每分钟补刀','场均小龙',\
                 '小龙控制率','场均大龙','大龙控制率','每分钟插眼','每分钟排眼','排眼效率','场均推塔数','场均被推塔数']
playerdata_head = ['名次','选手','战队','位置','出场次数','KDA','参团率','场均击杀','单场最高击杀','场均死亡','单场最高死亡','场均助攻','单场最高助攻',\
                   'GPM','CSPM','每分钟输出','输出占比','每分钟承受伤害','承受伤害占比','每分钟插眼数','每分钟排眼数']


In [3]:
def get_one_page(url):
    try:
        response = requests.get(url,headers =  headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

def parse_team(html):
    pattern = re.compile(
        '<tr>.*?<td>(.*?)</td>.*?text-hid">(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>'
        +'.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>'
         +'.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?</tr>', re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield item


def parse_player(html):
    pattern = re.compile(
        '<tr>.*?<td>(.*?)</td>.*?text-hid">(.*?)</td>.*?text-hid">(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>'
        +'.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?;">(.*?)</td>'
         +'.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?;">(.*?)</td>.*?</tr>', re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield item



def write_to_file_team(team_contents):
    with open('team.txt','w',encoding='utf-8') as f:
        for head in teamdata_head:
            f.write(head + '\t')
        f.write('\n') 
        for contents in team_contents:
            for content in contents:
                f.write(content + '\t')
            f.write('\n')    
        f.close()

def write_to_file_player(player_contents):
    with open('player.txt','w',encoding='utf-8') as f:
        for head in playerdata_head:
            f.write(head + '\t')
        f.write('\n') 
        
        for contents in player_contents:
            for content in contents:
                f.write(content + '\t')
            f.write('\n')    
        f.close()

def main():
    team = get_one_page(url_team)
    if team:
        print('采集战队页成功')
    for i in parse_team(team):
        print(i)
        write_to_file1(i)
    player = get_one_page(url_player)
    if player:
        print('采集选手页成功')
    for j in parse_player(player):
        print(j)
        write_to_file2(j)


In [11]:
team_html = get_one_page(url_team)
write_to_file_team(parse_team(team_html))

player_html = get_one_page(url_player)
write_to_file_player(parse_player(player_html))

In [13]:
team_data = pd.read_csv('team.txt',delim_whitespace=True)
team_data.head()

,名次,战队,KDA,场均击杀,场均死亡,每分钟伤害,一血率,场均时长,场均经济,每分钟经济,每分钟补刀,场均小龙,小龙控制率,场均大龙,大龙控制率,每分钟插眼,每分钟排眼,排眼效率,场均推塔数,场均被推塔数
0,1,OMG,2.0,10.2,16.6,1656,52.9%,32:31,55131,1695,31.65,1.4,34.1%,0.4,34.1%,3.13,1.38,41.4%,3.7,7.7
1,2,EDG,4.2,13.6,10.9,1815,72.7%,31:45,57752,1818,31.70,2.5,57.7%,0.7,52.6%,3.57,1.76,51.2%,6.5,5.0
2,3,IG,2.9,15.0,16.0,2155,38.6%,29:58,55157,1839,32.11,1.6,44.1%,0.6,48.2%,3.04,1.34,41.4%,5.5,5.8
3,4,LGD,2.4,10.8,15.4,1698,51.4%,33:18,56043,1682,30.88,2.3,53.1%,0.3,22.2%,3.34,1.44,43.4%,4.2,7.9
4,5,LNG,3.5,14.1,13.1,1805,34.9%,34:36,62999,1820,32.61,2.4,53.6%,0.9,62.5%,3.58,1.76,48.7%,6.6,6.1


In [14]:
player_data = pd.read_csv('player.txt',delim_whitespace=True)
player_data.head()

,名次,选手,战队,位置,出场次数,KDA,参团率,场均击杀,单场最高击杀,场均死亡,...,场均助攻,单场最高助攻,GPM,CSPM,每分钟输出,输出占比,每分钟承受伤害,承受伤害占比,每分钟插眼数,每分钟排眼数
0,1,KRYST4L,OMG,ADC,12,2.3,62.2%,2.5,7,3.2,...,4.9,10,402,8.74,424,25.7%,368,13.0%,0.48,0.45
1,2,cold,OMG,辅助,11,1.8,67.4%,0.5,3,3.0,...,4.9,11,237,1.52,129,7.8%,413,14.7%,1.63,0.30
2,3,World6,OMG,打野,20,1.9,72.2%,1.8,7,3.6,...,5.2,19,295,4.28,208,12.6%,751,25.0%,0.36,0.36
3,4,icon,OMG,中单,32,2.3,65.3%,3.0,8,3.0,...,3.9,16,385,8.40,446,26.7%,596,20.6%,0.42,0.17
4,5,Guoguo,OMG,中单,2,0.4,28.6%,0.0,0,2.5,...,1.0,2,318,8.22,289,22.2%,443,16.3%,0.55,0.09


In [4]:
plt.rc("font", size=14)	
plt.rcParams['savefig.dpi'] = 100 #图片像素	
plt.rcParams['figure.dpi'] = 100 #分辨率	
Data = pd.read_csv('LOL.csv')	
Data.rename(columns={'战队':'Team','攻击':'Attack',"法术":"Magic","防御":"Defence","团战":"Group","机动":"Flex","是否红方":"Is_Red","是否获胜":"IS_V"},inplace=True)	
Data.head()

,Team,Attack,Magic,Defence,Group,Flex,Is_Red,IS_V
0,EDG,46,48,64,86,68,0,0
1,EDG,50,52,62,78,50,0,0
2,EDG,46,64,54,70,62,1,1
3,EDG,54,54,50,74,70,1,1
4,EDG,58,52,60,68,58,0,1


In [39]:
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.commons.utils import JsCode
Data_1 = pd.pivot_table(Data,values='IS_V',index='Team',columns='Is_Red',aggfunc='sum')
Data_1.rename(columns={0:"NO",1:"YES"},inplace=True)
attr = Data_1.index.tolist()
v1 = Data_1['NO'].values.tolist()
v2 = Data_1['YES'].values.tolist()

bar = Bar()
bar.add_xaxis(attr)
bar.add_yaxis("Red", v2,stack="stack1")
bar.add_yaxis("Blue", v1,stack="stack1")
bar.set_global_opts(title_opts=opts.TitleOpts(title="Wins when selecting red or blue"))
bar.render_notebook()


In [56]:
from pyecharts.charts import Radar	
Data_2 = Data.groupby('Team').mean().stack()	
EDG_VALUE = [Data_2['EDG'][:5].values.tolist()]	
WE_VALUE = [Data_2['WE'][:5].values.tolist()]	
RNG_VALUE = [Data_2['RNG'][:5].values.tolist()]	
radar = Radar()	
c_schema= [ {"name": "Attack", "max": 55, "min": 53},	
           {"name": "Magic",   "max": 52.8, "min": 50},	
           {"name": "Defence", "max": 55, "min": 52},	
           {"name": "Group",   "max": 76,   "min":72},	
           {"name": "Flex",    "max": 68,    "min":64} ]	
radar.set_colors(["#4587E7"])
radar.add_schema(schema=c_schema,shape="circle",center=["50%", "50%"],radius="80%",
                 angleaxis_opts=opts.AngleAxisOpts(
                min_=0,
                max_=360,
                is_clockwise=False,
                interval=5,
                axistick_opts=opts.AxisTickOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
            ),
            radiusaxis_opts=opts.RadiusAxisOpts(
                min_=-4,
                max_=4,
                interval=2,
                splitarea_opts=opts.SplitAreaOpts(
                    is_show=True,
                    areastyle_opts=opts.AreaStyleOpts(opacity=1),
                )
            ),
            polar_opts=opts.PolarOpts(),
            splitarea_opt=opts.SplitAreaOpts(is_show=False),
            splitline_opt=opts.SplitLineOpts(is_show=False),
        )
#
radar.add("EDG", EDG_VALUE,linestyle_opts=opts.LineStyleOpts(width=2),color="red",areastyle_opts=opts.AreaStyleOpts(opacity=0.1))	
radar.add("WE", WE_VALUE,color="blue",areastyle_opts=opts.AreaStyleOpts(opacity=0.1))
radar.add("RNG", RNG_VALUE,color="black",areastyle_opts=opts.AreaStyleOpts(opacity=0.1))
radar.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
radar.render_notebook()

In [62]:
from pyecharts.charts import Bar
from pyecharts.globals import ThemeType
Data_3 = Data.groupby('Team').sum()	
Data_4 = Data_3['IS_V']	
Data_4 = Data_4.sort_values(ascending=False)	
attr = Data_4.index.values.tolist()	
v = Data_4.values.tolist()	
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
bar.add_xaxis(attr)
bar.add_yaxis("战队胜场排行榜",v)	
bar.render_notebook()

In [87]:
from pyecharts.charts import Pie	
from pyecharts import options as opts
import numpy as np

Data_7 = Data.groupby('Team').sum()	
Index = Data_7.index	
ALL_COUNT = np.array([len(Data[Data.Team == i ]) for i in Index])	
ALL_WIN =Data_7.IS_V.values	
WIN_PER = ALL_WIN/ALL_COUNT	
LOS_PER = 1 - WIN_PER	
Data_8 = pd.DataFrame(index=Index,columns=['WIN_PER','LOS_PER'])	
Data_8['WIN_PER'] = WIN_PER*100	
Data_8['LOS_PER'] = LOS_PER*100	
Data_8.sort_values(by='WIN_PER',ascending=False,inplace=True)	

pie = Pie()	
pie.set_global_opts(title_opts=opts.TitleOpts(pos_left = 'center'))
pie.set_series_opts(label_opts=opts.LabelOpts(position = 'center',is_show = True, color = None))



pie.add('', [list(z) for z in zip(["EDG win", "EDG lose"], Data_8.iloc[0].values.tolist())], center=['10%', '30%'], radius=[18, 24])	
pie.add('', [list(z) for z in zip(["RNG win", "RNG lose"], Data_8.iloc[1].values.tolist())], center=['30%', '30%'], radius=[18, 24])
pie.add('', [list(z) for z in zip(["WE win", "WE lose"], Data_8.iloc[2].values.tolist())], center=['50%', '30%'], radius=[18, 24])
pie.add('', [list(z) for z in zip(["IG win", "IG lose"], Data_8.iloc[3].values.tolist())], center=['70%', '30%'], radius=[18, 24])
pie.add('', [list(z) for z in zip(["OMG win", "OMG lose"], Data_8.iloc[4].values.tolist())], center=['90%', '30%'], radius=[18, 24])
pie.add('', [list(z) for z in zip(["NB win", "NB lose"], Data_8.iloc[5].values.tolist())], center=['10%', '70%'], radius=[18, 24])
pie.add('', [list(z) for z in zip(["JDG win", "JDG lose"], Data_8.iloc[6].values.tolist())], center=['30%', '70%'], radius=[18, 24])
pie.add('', [list(z) for z in zip(["SNG win", "SNG lose"], Data_8.iloc[7].values.tolist())], center=['50%', '70%'], radius=[18, 24])
pie.add('', [list(z) for z in zip(["LGD win", "LGD lose"], Data_8.iloc[8].values.tolist())], center=['70%', '70%'], radius=[18, 24])
pie.add('', [list(z) for z in zip(["DAN win", "DAN lose"], Data_8.iloc[9].values.tolist())], center=['90%', '70%'], radius=[18, 24])

pie.render_notebook()